In [1]:
from corpus_characterizer import generator_chunker
import numpy as np
import os
# from sklearn.metrics import mean_squared_error,mean_absolute_error, median_absolute_error, mean_squared_log_error, explained_variance_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, \
    r2_score
import pandas as pd
from scipy.stats import describe, kurtosistest, skewtest, normaltest
from Conv1D_LSTM_Ensemble import pair_generator_1dconv_lstm_bagged
from Conv1D_ActivationSearch_BigLoop import pair_generator_1dconv_lstm #NOT BAGGED
from AuxRegressor import create_testing_set, create_training_set
import matplotlib.pyplot as plt 

# @@@@@@@@@@@@@@ RELATIVE PATHS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Base_Path = "./"
image_path = "./images/"
train_path = "./train/"
test_path = "./test/"
analysis_path = "./analysis/"
chunker_path = analysis_path + "chunker/"
preds_path = analysis_path + "preds/"

#create the data-pair filenames (using zip), use the helper methods
train_set_filenames = create_training_set()
test_set_filenames = create_testing_set()
print(train_set_filenames)
print(test_set_filenames)

# for files in test_set_filenames:
#     data_load_path = test_path + '/data/' + files[0]
#     label_load_path = test_path + '/label/' + files[1]
#     # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
#     test_array = np.load(data_load_path)
#     test_label_array = np.load(label_load_path)[:, 1:]

Using Theano backend.


after shuffling: [('sequence_3a_705_7_fv1c.npy', 'sequence_3a_705_7_label_fv1c.npy'), ('sequence_1k_331_2_fv1c.npy', 'sequence_1k_331_2_label_fv1c.npy'), ('sequence_1l_424_5_fv1c.npy', 'sequence_1l_424_5_label_fv1c.npy'), ('sequence_1k_300_1_fv1c.npy', 'sequence_1k_300_1_label_fv1c.npy'), ('sequence_2c_510_1_fv1c.npy', 'sequence_2c_510_1_label_fv1c.npy'), ('sequence_1d_121_2_fv1c.npy', 'sequence_1d_121_2_label_fv1c.npy'), ('sequence_1c_101_2_fv1c.npy', 'sequence_1c_101_2_label_fv1c.npy'), ('sequence_1c_72_3_fv1c.npy', 'sequence_1c_72_3_label_fv1c.npy'), ('sequence_1c_112_3_fv1c.npy', 'sequence_1c_112_3_label_fv1c.npy'), ('sequence_3a_699_1_fv1c.npy', 'sequence_3a_699_1_label_fv1c.npy'), ('sequence_1d_123_4_fv1c.npy', 'sequence_1d_123_4_label_fv1c.npy'), ('sequence_2b_475_6_fv1c.npy', 'sequence_2b_475_6_label_fv1c.npy'), ('sequence_3a_703_5_fv1c.npy', 'sequence_3a_703_5_label_fv1c.npy'), ('sequence_2c_532_3_fv1c.npy', 'sequence_2c_532_3_label_fv1c.npy'), ('sequence_3a_710_12_fv1c.npy', 

In [2]:
score_per_ic_df = pd.read_csv(analysis_path+'sn_ic_grouped_by_ic.csv',index_col=0)
score_per_loadseq_df = pd.read_csv(analysis_path+'srls_grouped_by_ls.csv',index_col=0)
print(score_per_ic_df.index)
print(score_per_loadseq_df.index)

Index([u'[ 0.410625  1.335625  1.515625  2.130625]',
       u'[ 0.420625  1.345625  1.425625  2.215625]',
       u'[ 0.490625  1.365625  1.415625  2.253125]',
       u'[ 0.610625  1.335625  1.515625  2.130625]',
       u'[ 0.620625  1.335625  1.515625  2.130625]',
       u'[ 0.725   1.475   1.525   2.2375]'],
      dtype='object', name=u'init_cond')
Index([u'[  1.028  10.28   10.28    0.75    0.75   10.28   10.28    1.028]',
       u'[  1.028  11.28   11.28    1.      1.     10.28   10.28    1.028]',
       u'[  1.028  12.28   12.28    1.028   1.028  17.28   17.28    1.028]',
       u'[  1.028  13.28   13.28    1.25    1.25   13.28   13.28    1.028]',
       u'[  1.028  14.28   14.28    0.75    0.75   16.28   16.28    1.028]',
       u'[  1.028  15.28   15.28    1.75    1.75   10.28   10.28    1.028]',
       u'[  1.428  12.28   12.28    1.428   1.428  16.28   16.28    1.428   1.428\n  14.28   14.28    0.75    0.75   14.28   14.28    1.428   1.428  14.28\n  14.28    0.75    0.75   16.2

In [7]:
seqname_initcond_df = pd.read_csv(analysis_path + 'combined_seqname_initcond_dict.csv')
seqname_loadseq_df = pd.read_csv(analysis_path + 'seqname_score_loadseq_TEST.csv')
#loadseq_initcond_pair_df = pd.merge(seqname_initcond_df,seqname_loadseq_df,how='inner',left_index=True,right_index=True)
loadseq_initcond_pair_df = pd.merge(seqname_initcond_df,seqname_loadseq_df,how='inner',left_on='seq_name',right_on='seq_name')

print(loadseq_initcond_pair_df.head(1))

                 seq_name                          init_cond   mse_avg  \
0  sequence_1d_168_9_fv1c  [ 0.725   1.475   1.525   2.2375]  0.093251   

    mse_med  mse_stdev   mae_avg   mae_med  mae_stdev  mae_normd_dev  \
0  0.013695   0.129213  0.141744  0.012286   0.187901            NaN   

     mape_avg    mape_med  mape_stdev  msle_avg  msle_med  msle_stdev  \
0  483.399914  556.930833  174.561483  0.063196  0.058232    0.009147   

   loadseq_id                                            loadseq  
0           0  [  1.428  14.28   14.28    1.75    1.75   14.2...  


In [8]:
loadseq_initcond_pair_df.to_csv(analysis_path + 'loadseq_initcond_pairs.csv')

In [12]:
grouped = loadseq_initcond_pair_df.groupby(['loadseq','init_cond']).count()
pd.DataFrame(grouped).to_csv(analysis_path+'loadseq_initcond_pairs_count.csv')